## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.


In [1]:
import pandas as pd
data = pd.read_csv('/Users/eha19/OneDrive/Desktop/progect/data.csv')
print(data.head(10))
data.info()
data.describe()

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

В таблице представлено 12 столбцов и 21525 строк. При выводе первых 10 столбцов видные некоторые ошибки в данных, например, разный регистр в столбце education или отрицательные значения в столбце days_employed. При просмотре информации о таблице видны пропущенные значения в столбцах days_employed и total_income, можно заметить, что число пропущенных значений в данных столбцах равно между собой.

### Шаг 2. Предобработка данных

In [2]:
# КОД РЕВЬЮВЕРА

data[['days_employed', 'total_income', 'dob_years']].describe()

,days_employed,total_income,dob_years
count,19351.000000,1.935100e+04,21525.000000
mean,63046.497661,1.674223e+05,43.293380
std,140827.311974,1.029716e+05,12.574584
min,-18388.949901,2.066726e+04,0.000000
25%,-2747.423625,1.030532e+05,33.000000
50%,-1203.369529,1.450179e+05,42.000000
75%,-291.095954,2.034351e+05,53.000000
max,401755.400475,2.265604e+06,75.000000


### Обработка пропусков

In [3]:
print(data.isnull().sum())
exp_median = data.groupby('income_type')['days_employed'].transform('median')
data['days_employed'] = data['days_employed'].fillna(exp_median)
group_emloyed = data.groupby('income_type')['days_employed']
salary_median = data.groupby('income_type')['total_income'].transform('median')
data['total_income'] = data['total_income'].fillna(salary_median)
data['dob_years'] = data['dob_years'].replace(0, data['dob_years'].median())
print(group_emloyed.describe())
def years_employed(days_employed):
    if days_employed > 300000:
        return days_employed / 8760
    return abs(days_employed / 365)
data['years_employed'] = data['days_employed'].apply(years_employed)
group_emloyed_2 = data.groupby('income_type')['years_employed']
print(group_emloyed_2.describe())
print(data.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
                   count           mean           std            min  \
income_type                                                            
безработный          2.0  366413.652744  40855.478519  337524.466835   
в декрете            1.0   -3296.759962           NaN   -3296.759962   
госслужащий       1459.0   -3328.308350   2652.713016  -15193.032201   
компаньон         5085.0   -2055.165652   1950.764283  -17615.563266   
пенсионер         3856.0  365025.963652  19909.115430  328728.720605   
предприниматель      2.0    -520.848083      0.000000    -520.848083   
сотрудник        11119.0   -2251.736421   2201.767262  -18388.949901   
студент              1.0    -578.751554

### Вывод

Пропуска типа NaN обнаружены и заменены в столбцах days_employed и total_income. Пропущенные значения в столбцах days_employed и total_income совпадают построчно, скорее всего это связано с тем, что заемщик не предоставил данные о работе. Так же встречаются значения 0 в столбце dob_years, скорее всего заемщик не предоставил данные о возрасте. Считаю, в данном случае, верным не удалять пустые значения, а заменить их на медианные. В столбце days_employed часто встречаются отрицательные значения, предположу, что исправить их можно просто убрав минус. В столбце dob_years присутствуют нулевые значения возраста, они также заменены на медианные значения.

### Замена типа данных

In [4]:
data['days_employed'] = data['days_employed'].astype('int64')
data['dob_years'] = data['dob_years'].astype('int64')
data['total_income'] = data['total_income'].astype('int64')
data['years_employed'] = data['years_employed'].astype('int64')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
years_employed      21525 non-null int64
dtypes: int64(8), object(5)
memory usage: 2.1+ MB


### Вывод

Был присвоен целочисленный тип данных для столбцов, в каждом из них дробная часть не влияет на результат исследования. Использовался метод astype(), т.к. нужен был перевод в целочисленный тип данных

### Обработка дубликатов

In [5]:
print(data['children'].value_counts()) #(разобраться с -1 и 20)
print(data['education'].value_counts()) # привести к одному формату
print(data['education_id'].value_counts())# все верно
print(data['family_status'].value_counts()) # все верно
print(data['family_status_id'].value_counts()) # все верно
print(data['gender'].value_counts()) #(1 с неопределенным полом)
print(data['income_type'].value_counts())# все верно
print(data['debt'].value_counts()) #(все верно)
print(data['purpose'].value_counts()) #(объединить категории)
print(data['years_employed'].min())
print(data['years_employed'].max()) 
print(data['dob_years'].value_counts())
print(data['total_income'].min())
print(data['total_income'].max()) # значения в пределах нормы
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender'] = data['gender'].replace('XNA', 'F')
#проверка:
print(data['children'].value_counts())
print(data['education'].value_counts())
print(data['family_status'].value_counts())
print(data['gender'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64
1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64
женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64
F      14236
M       7288
XNA    

In [6]:
print(data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop = True)
print(data.duplicated().sum())

72
0


### Вывод

Для категориальных переменных (а также для столбца dob_years, т.к. общее количество значений там невелико) для поиска дубликатов применен метод value_counts(), т.к. результатов немного и информация получается в более удобным для обзора виде, чем при применении метода unique(), для количественных переменных брались минимальные и максимальные значения, т.к. формат каждого из таких столбцов int, то можно предположить, что все значения целочисленные, и оценить данные столбцы по крайним значениям. Дубликаты в столбце education заменены приведением всех значений к строчным буквам, в столбце family_status проведена аналогичная процедура. в остальных столбцах проведена замена значений, т.к. их немного данная операция применена напрямую к каждому значению. Дубликаты появились в столбце education, возможно, из-за неправильного ввода значений в строки, предположу что минусы появились при вводе тире. 

### Лемматизация

In [7]:
from pymystem3 import Mystem
m = Mystem() 
for i in data['purpose'].unique():
    lemmas = m.lemmatize(i)
    print(lemmas)

['покупка', ' ', 'жилье', '\n']
['приобретение', ' ', 'автомобиль', '\n']
['дополнительный', ' ', 'образование', '\n']
['сыграть', ' ', 'свадьба', '\n']
['операция', ' ', 'с', ' ', 'жилье', '\n']
['образование', '\n']
['на', ' ', 'проведение', ' ', 'свадьба', '\n']
['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n']
['покупка', ' ', 'недвижимость', '\n']
['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n']
['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n']
['недвижимость', '\n']
['строительство', ' ', 'недвижимость', '\n']
['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n']
['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n']
['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n']
['жилье', '\n']
['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n']
['автомобиль', '\n']
['заниматься', ' ', 'образование'

### Вывод

При получении лемм получилось увидеть 5 ключевых слов: жилье, недвижимость, свадьба, автомобиль, образование, из них можно составить 4 категории (жилье и недвижимость можно объединить в одну категорию) и привести каждое значение из столбца purpose к какой-либо категории.

### Категоризация данных

In [ ]:
def purpose_group(purpose):
    lemmas = m.lemmatize(purpose)
    if 'недвижимость' in lemmas:
        return 'операции с недвижимостью'
    if 'жилье' in lemmas:
        return 'операции с недвижимостью'
    if 'свадьба' in lemmas:
        return 'проведение свадьбы'
    if 'образование' in lemmas:
        return 'получение образования'
    return 'сделка с автомобилем'
data['purpose_group'] = data['purpose'].apply(purpose_group)
print(data['purpose_group'].value_counts())    
def children_group(children):
    if children == 0:
        return 'нет детей'
    if children == 1:
        return '1 ребенок'
    if children == 2:
        return '2 детей'
    return '3 и более детей'
data['children_group'] = data['children'].apply(children_group)
print(data['children_group'].value_counts())
def total_income_group(total_income):
    if total_income < 100000:
        return 'Доход менее 100 000 р.'
    if total_income < 150000:
        return 'Доход от 100 до 150 тысяч р.'
    if total_income < 200000:
        return 'Доход от 150 до 200 тысяч р.'
    if total_income < 300000:
        return 'Доход от 200 до 300 тысяч р.'
    return 'Доход свыше 300 000 р.'
data['total_income_group'] = data['total_income'].apply(total_income_group)
print(data['total_income_group'].value_counts())    

### Вывод

Разобьем нужные нам столбцы на категории: purpose на 4 группы по ключевым словам, полученным при лемматизации, children: количество измерений с числом детей большим 3 невелико по сравнению с остальными пунктами, поэтому данные измерения можно объединить в одну категорию, столбец total_income разделим по размеру дохода, разделение на от 100 до 150 и от 150 до 200 связано с тем, что при объединении слишком большой процент всех значений оказывается в данной категории.

### Шаг 3. Ответьте на вопросы

In [ ]:
import plotly.express as px

def pivot(data, index):
    pivot = data.pivot_table(index = index, values = 'debt', aggfunc = ['count', 'sum', 'mean']).reset_index()
    pivot.columns = [index, 'Кол-во клиентов', 'Кол-во должников', '% невозврата']
    display(pivot.sort_values('% невозврата', ascending = False))
    
    
    
    fig = px.bar(pivot, x = index, y = '% невозврата', color = '% невозврата', title = '% невозврата по ' + index)
    fig.show()

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
pivot(data, 'children_group')

### Вывод

По полученным данным можно увидеть, что по мере увеличения количества детей до 2 доля невозвращенных в срок кредитов увеличивается, но при наличие 3 и более детей доля становится минимальной, возможно это связано с тем, что количество человек с большим количеством детей, берущих кредиты, весьма не велико и выборка нерепрезентативна). В целом такая зависимость наблюдается, если отбросить последнюю категорию.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
pivot(data, 'family_status')

### Вывод

В данном случае видно что доля невозврата среди категории никогда не состоящих в браке наиболее максимально, но не понятно, почему доля вдовцов и разведенных настолько меньше и даже меньше доли для людей находящихся в браке. По моему мнению, в данном случае это можно объснить тем, что в среднем разведенные и вдовцы/вдовы старше и имеют больший доход. По итогу можно сделать вывод о том что зависимость от семейного статуса есть, но она неочевидна.

In [ ]:
pivot(data, 'total_income_group')

### Вывод

Первым выводом является что разделять категорию доход от 100 до 200 тыс. р. было необязательным, т.к. доля невозврата в них практически идентична. В целом видно, что при большем доходе доля возвратов выше, максимальная доля невозвратов фиксируется в значении от 100 до 200 тыс. р. В целом это может быть связано с размерами кредитов. Намного более информативным было бы сравнение результатов по доле от дохода, ежемесячно выплачиваемого в счет погашения кредита. В целом зависимость можно охарактеризовать как то, что при доходе меньшем чем 200 000 руб. доля невозврата больше на 15-20%, чем при доходе выше 200 тыс.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
pivot(data, 'purpose_group')

### Вывод

Наиболее ожидаемым результатом было увидеть минимальную долю возвратов от кредитов на свадьбы, т.к. свадьба не ведет к повышению дохода, либо к получению материальной ценности, такой как автомобиль либо недвижимость и макимальную долю по операциям с недвижимостью. По итогу получаем, что кредиты сделки с автомобилем и получение образования чаще не возвращаются, чем кредиты на недвижимость и свадьбы. Кредиты с недвижимостью ожидаемо имеют наименьшую долю не возврата. В целом получена статистика, объяснить ее можно тем, что получение образования ведет к увеличению дохода только через несколько лет, покупка автомобиля в кредит в некоторых случаях осуществляется если автомобиль не соответствует доходу заемщика, в таких случаях возможен более частый невозврат средств.

### Шаг 4. Общий вывод

В результате сравнивается общая доля непогашенных кредитов и доля для каждой из категорий, также для наглядности показано относительное сравнение категорий со средним для большей наглядности. В целом можно охарактеризовать, что при рассмотре зависимостей от данных категорий нет каких-либо особо крупных выпадений из общей статистики. Макимальное отличие от среднего составляет 20%, к тому же в 3 и 4 случаях категория с наибольшим количеством измерений оказывалась с наименьшей долей невозвратов, что тянуло значения средней доли вниз. Это также показывает, что чаще кредиты дают при тех условиях, когда доля возвратов наименьшая. В случае зависимости от уровня дохода, по-моему мнению она имеет небольшое влияние, т.к. кредит обычно берется, на то, на что у заемщика не хватает личных средств, соответственно чем больше доход, тем больше и размер кредита. В целом вывод в том, что по каждой рассматриваемой категории прослеживается определенное влияние, но оно не оказывается особо крупным.